## Load Library

In [1]:
import math
import time
import itertools

import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline

## Read File

In [2]:
inFile = open("tsp.txt", "r")

## Convert File to Pandas

In [3]:
arr_list = []

inFile.seek(0)
for line in inFile.readlines():
    line = line[:-1]
    arr_list.append(line.split(" "))
    
df = pd.DataFrame(arr_list, columns = ["Name", "X", "Y"])

# Convert Value From String to int
df["X"] = df["X"].astype(int)
df["Y"] = df["Y"].astype(int)

df

,Name,X,Y
0,Depot,265,143
1,Parcel1,253,278
2,Point2,439,148
3,Parcel3,458,304
4,Point4,609,230
5,Parcel5,597,101


In [4]:
def distance(p1, p2):
    return math.sqrt((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2)

In [5]:
class Point:
    def __init__(self, name, x, y):
        self.name = name
        self.x = x
        self.y = y
        
    def toString(self):
        return f"Name: {self.name}, X Pos: {self.x}, Y Pos: {self.y}"

In [6]:
def create_point(ind):
    row = df.iloc[ind, :]
    return Point(row.Name, row.X, row.Y)

In [7]:
def add_edge_to_graph(G, e1, e2, w):
    G.add_edge(e1, e2, weight=w)

## Create Point Array

In [8]:
pt_arr = []

inFile.seek(0)
for line in inFile.readlines():
    line = line[:-1]
    tmp_arr = line.split(" ")
    
    pt_name = tmp_arr[0]
    x_pos = int(tmp_arr[1])
    y_pos = int(tmp_arr[2])
    
    pt = Point(pt_name, x_pos, y_pos)
    
    pt_arr.append(pt)
    
name_arr = list(map(lambda x : x.name, pt_arr))
name_arr

['Depot', 'Parcel1', 'Point2', 'Parcel3', 'Point4', 'Parcel5']

## Create Edge Array

In [9]:
adj_mat = []

name_arr = list(map(lambda x : x.name, pt_arr))

dist_val = 0

for pt in pt_arr:
    
    name = pt.name
    
    tmp_arr = []
    
    for pt2 in pt_arr:
        dist_val = round(distance(pt, pt2), 3)
        if pt.name == pt2.name or ("Depot" in pt.name and "Point" in pt2.name) or ("Point" in pt.name and "Point" in pt2.name):
            dist_val = float('inf')
        tmp_arr.append(dist_val)
    
    adj_mat.append(tmp_arr)
    
adj_mat

[[inf, 135.532, inf, 251.336, inf, 334.646],
 [135.532, inf, 226.927, 206.642, 359.221, 386.866],
 [174.072, 226.927, inf, 157.153, inf, 164.842],
 [251.336, 206.642, 157.153, inf, 168.158, 246.028],
 [354.831, 359.221, inf, 168.158, inf, 129.557],
 [334.646, 386.866, 164.842, 246.028, 129.557, inf]]

In [10]:
def gen_adj_mat(adj_mat, column_arr):
    tmp_adj_mat = []
    for row_ind in range(len(adj_mat)):
        if row_ind in column_arr:
            tmp_list = []
            for col_ind in column_arr:
                tmp_list.append(adj_mat[row_ind][col_ind])
            tmp_adj_mat.append(tmp_list)
            
    return tmp_adj_mat

In [11]:
gen_adj_mat(adj_mat, [0, 2, 3, 4, 5])

[[inf, inf, 251.336, inf, 334.646],
 [174.072, inf, 157.153, inf, 164.842],
 [251.336, 157.153, inf, 168.158, 246.028],
 [354.831, inf, 168.158, inf, 129.557],
 [334.646, 164.842, 246.028, 129.557, inf]]

## Create Path

In [12]:
class Path:
    def __init__(self, path, cost):
        self.path = path
        self.cost = cost
        
    def __lt__(self, other):
        return self.cost <= other.cost
        
    def __str__(self):
        return f"Shortest Path:{self.path}\nMinimum Cost: {self.cost}"
    
    def __eq__(self, other):
        return self.path == other.path and self.cost == other.cost
    
    def getNodePath(self, name_arr):
        tmp_arr = list(map(lambda x : name_arr[x], self.path))
        return "->".join(tmp_arr)

## Create TSP Matrix

In [13]:
def held_karp(dists, column_arr):
    """
    Implementation of Held-Karp, an algorithm that solves the Traveling
    Salesman Problem using dynamic programming with memoization.
    Parameters:
        dists: distance matrix
    Returns:
        A tuple, (cost, path).
    """
    n = len(dists)

    # Maps each subset of the nodes to the cost to reach that subset, as well
    # as what node it passed before reaching this subset.
    # Node subsets are represented as set bits.
    C = {}

    # Set transition cost from initial state
    for k in range(1, n):
        C[(1 << k, k)] = (dists[0][k], 0)

    # Iterate subsets of increasing length and store intermediate results
    # in classic dynamic programming manner
    for subset_size in range(2, n):
        for subset in itertools.combinations(range(1, n), subset_size):
            # Set bits for all nodes in this subset
            bits = 0
            for bit in subset:
                bits |= 1 << bit
            
            # print(bits, subset)

            # Find the lowest cost to get to this subset
            for k in subset:
                prev = bits & ~(1 << k)

                res = []
                for m in subset:
                    if m == 0 or m == k:
                        continue
                    res.append((C[(prev, m)][0] + dists[m][k], m))
                    
                # print((bits, k))
                C[(bits, k)] = min(res)

    # Calculate optimal cost
    res = []
    for k in range(1, n):
        res.append((C[(bits, k)][0] + dists[k][0], k))
    opt, parent = min(res)

    # Backtrack to find full path
    path = ["Depot"] * (n + 1)
    for i in range(n - 1, 0, -1):
        tmp_val = parent
        path[i] = column_arr[tmp_val]
        _, parent = C[(bits, tmp_val)]
        bits = bits & ~(1 << tmp_val)
        
    return opt, path

In [14]:
# held_karp(adj_mat, name_arr)

In [15]:
# parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
# point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]

# hk_path = Path([], float('inf'))

# final_start = time.process_time()

# combo_set = set(itertools.combinations(parcel_arr, len(point_arr)))
# total, ind = len(combo_set), 1

# print(f"Total Combinations: {total}")
# for parcel_set in combo_set:
    
#     tmp_list = [0] + [*parcel_set] + point_arr
#     tmp_list = sorted(tmp_list)
    
#     tmp_adj_mat = gen_adj_mat(adj_mat, tmp_list)
    
#     tmp_list = list(map(lambda x : name_arr[x], tmp_list))
    
#     start = time.process_time()
#     val, best_path = held_karp(tmp_adj_mat, tmp_list)
#     print(f"Time Taken (Held-Karp): {time.process_time() - start}")
    
#     hk_path = min(hk_path, Path(best_path, val))
    
#     print(f"Progress: {round(ind / total * 100.0, 2)}%\n")
    
#     ind += 1
    
# print(f"Final Time Taken (Held-Karp): {time.process_time() - final_start}")
# print(hk_path)

## Optimized Held Karp

In [16]:
# parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
# point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]

# start = time.process_time()
# subset_arr, route_arr = [], []
# pN, n = len(point_arr), len(name_arr)
# for subset_size in range(2, 2 * pN):
#     for subset in itertools.combinations(range(1, n), subset_size):
#         subset_arr.append(subset)
        
# for subset in itertools.combinations(parcel_arr, len(point_arr)):
#     tmp_arr = sorted([*subset] + point_arr)
#     subset_arr.append(tuple(tmp_arr))
#     route_arr.append(sum([1 << elem for elem in tmp_arr]))
    
# print(subset_arr)

# print(time.process_time() - start)

In [17]:
parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]

start = time.process_time()

subset_arr = []

for k in range(1, len(point_arr) + 1):
    for gSubset in itertools.combinations(parcel_arr, k):
        for l in range(k - 1, k + 1):
            if l == 0:
                continue
            for rSubset in itertools.combinations(point_arr, l):
                subset_arr.append(([*gSubset], [*rSubset]))
                print(([*gSubset], [*rSubset]))
            
                
# route_arr = []
# for subset in itertools.combinations(parcel_arr, len(point_arr)):
#     tmp_arr = sorted([*subset] + point_arr)
#     route_arr.append(sum([1 << elem for elem in tmp_arr]))
                
# subset_arr = sorted(subset_arr, key = lambda x : len(x[0]) + len(x[1]))
# print(subset_arr)

# print(route_arr)

([1], [2])
([1], [4])
([3], [2])
([3], [4])
([5], [2])
([5], [4])
([1, 3], [2])
([1, 3], [4])
([1, 3], [2, 4])
([1, 5], [2])
([1, 5], [4])
([1, 5], [2, 4])
([3, 5], [2])
([3, 5], [4])
([3, 5], [2, 4])


In [36]:
def held_karp_2(dists, parcel_arr, point_arr, name_arr):
    """
    Implementation of Held-Karp, an algorithm that solves the Traveling
    Salesman Problem using dynamic programming with memoization.
    Parameters:
        dists: distance matrix
    Returns:                             
        A tuple, (cost, path).
    """
    n = len(dists)

    # Maps each subset of the nodes to the cost to reach that subset, as well
    # as what node it passed before reaching this subset.
    # Node subsets are represented as set bits.
    C = {}
    
    subset_arr, route_arr = [], []
    for k in range(1, len(point_arr) + 1):
        for gSubset in itertools.combinations(parcel_arr, k):
            for l in range(k - 1, k + 1):
                if l == 0:
                    continue
                for rSubset in itertools.combinations(point_arr, l):
                    subset_arr.append(([*gSubset], [*rSubset]))
                    
                    if len(gSubset) + len(rSubset) == 2 * len(point_arr):
                        bits = 0
                        for bit in gSubset:
                            bits |= 1 << bit
            
                        for bit in rSubset:
                            bits |= 1 << bit
                        
                        print(bits)
                        
                        route_arr.append(bits)
                
    subset_arr = sorted(subset_arr, key = lambda x : len(x[0]) + len(x[1]))

    # Set transition cost from initial state
    for k in range(1, n):
        C[(1 << k, k)] = (dists[0][k], 0)

    # Iterate subsets of increasing length and store intermediate results
    # in classic dynamic programming manner
    for subset in subset_arr:
        gSubset, rSubset = subset
        # Set bits for all nodes in this subset
        bits = 0
        
        for bit in gSubset:
            bits |= 1 << bit
            
        for bit in rSubset:
            bits |= 1 << bit
        
        subset2 = []
        
        if len(gSubset) == len(rSubset):
            # The last node FROM THE PREVIOUS SUBSET will be a red point
            subset2 = gSubset
        else:
            # The last node FROM THE PREVIOUS SUBSET will be a green point
            subset2 = rSubset
            
        subset1 = [0] * (len(gSubset) + len(rSubset))
        
        ind = 0
        for elem in gSubset:
            subset1[ind] = elem
            ind += 1
            
        for elem in rSubset:
            subset1[ind] = elem
            ind += 1

        # Find the lowest cost to get to this subset
        for k in subset1:
            prev = bits & ~(1 << k)

            opt, parent = float('inf'), 0
            for m in subset2:     
                # Check if Key Exist
                key = (prev, m)
                if key in C:
                    tmp_opt = C[key][0] + dists[m][k]
                    if tmp_opt <= opt:
                        opt = tmp_opt
                        parent = m
                
            C[(bits, k)] = (opt, parent)
            
    for k, v in C.items():
        print(k, v)
                
    bits = 0
    opt = float('inf')
    parent = 0

    # Calculate optimal cost
    for route_bits in route_arr:
        for k in point_arr:
            tmp_opt = C[(route_bits,k)][0] + dists[k][0]
            
            if tmp_opt <= opt:
                opt = tmp_opt
                parent = k
                bits = route_bits

    # Backtrack to find full path
    path = ["Depot"] * (2 * len(point_arr) + 2)
    for i in range(2 * len(point_arr), 0, -1):
        tmp_val = parent
        path[i] = name_arr[tmp_val]
        _, parent = C[(bits, tmp_val)]
        bits = bits & ~(1 << tmp_val)
        
    return opt, path

In [37]:
parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]
print(parcel_arr, point_arr)

[1, 3, 5] [2, 4]


In [38]:
parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]
start = time.process_time()
val, best_path = held_karp_2(adj_mat, parcel_arr, point_arr, name_arr)
hk2_path = Path(best_path, val)
print(hk2_path)
print(time.process_time() - start)

30
54
60
(2, 1) (135.532, 0)
(4, 2) (inf, 0)
(8, 3) (251.336, 0)
(16, 4) (inf, 0)
(32, 5) (334.646, 0)
(6, 1) (inf, 0)
(6, 2) (362.459, 1)
(18, 1) (inf, 0)
(18, 4) (494.75300000000004, 1)
(12, 3) (inf, 0)
(12, 2) (408.48900000000003, 3)
(24, 3) (inf, 0)
(24, 4) (419.494, 3)
(36, 5) (inf, 0)
(36, 2) (499.48800000000006, 5)
(48, 5) (inf, 0)
(48, 4) (464.203, 5)
(14, 1) (635.416, 2)
(14, 3) (519.612, 2)
(14, 2) (inf, 0)
(26, 1) (778.715, 4)
(26, 3) (662.9110000000001, 4)
(26, 4) (inf, 0)
(38, 1) (726.4150000000001, 2)
(38, 5) (527.301, 2)
(38, 2) (inf, 0)
(50, 1) (823.424, 4)
(50, 5) (624.3100000000001, 4)
(50, 4) (inf, 0)
(44, 3) (656.6410000000001, 2)
(44, 5) (573.331, 2)
(44, 2) (inf, 0)
(56, 3) (632.361, 4)
(56, 5) (549.051, 4)
(56, 4) (inf, 0)
(30, 1) (inf, 0)
(30, 3) (inf, 0)
(30, 2) (820.0640000000001, 3)
(30, 4) (687.77, 3)
(54, 1) (inf, 0)
(54, 5) (inf, 0)
(54, 2) (789.152, 5)
(54, 4) (656.8580000000001, 5)
(60, 3) (inf, 0)
(60, 5) (inf, 0)
(60, 2) (713.893, 5)
(60, 4) (702.888, 

In [29]:
# print(hk_path == hk2_path)